This notebook demonstrates integration of functionality from SNT with Python through pyimagej.  
The context is the computation and rendering of the convex hull enclosing all axonal/dendritic terminal nodes.    

In [1]:
import sys
#!conda install --yes --prefix {sys.prefix} -c conda-forge pyimagej openjdk=8 scipy

In [2]:
import imagej
import numpy as np
from scipy.spatial import ConvexHull
from collections import defaultdict

In [3]:
# Initialize Fiji with GUI support.
ij = imagej.init(r'C:\Users\arshadic\Downloads\fiji-win64_unstable\Fiji.app', headless=False)

from jnius import autoclass, cast

In [4]:
# Import relevant SNT (Java) classes.
HashSet = autoclass('java.util.HashSet')
PointInImage = autoclass('sc.fiji.snt.util.PointInImage')
MouseLightLoader = autoclass('sc.fiji.snt.io.MouseLightLoader')
Tree = autoclass('sc.fiji.snt.Tree')
TreeAnalyzer = autoclass('sc.fiji.snt.analysis.TreeAnalyzer')
Viewer = autoclass('sc.fiji.snt.viewer.Viewer3D')

In [5]:
def convex_hull(tree):
    """Computes the convex hull of the input Tree
    and return a Java ArrayList containing the vertices 
    representing the hull.

    Returns:
        verts_java (ArrayList): The list containing the hull vertices.
    Args:
        tree (Tree): The SNT Tree object."""

    analyzer = TreeAnalyzer(tree)
    # Extract the end points from the tree as a java HashSet.
    tips_java_set = analyzer.getTips()
    # Convert to array to maintain ordering.
    tips_java_array = tips_java_set.toArray()
    
    compartment_dict = defaultdict(list)
    for t in tips_java_array:
        annotation = t.getAnnotation()
        if annotation is not None:
            compartment_dict[annotation.name()].append(t)
    
    max_compartment = max(compartment_dict, key= lambda x: len(compartment_dict[x]))
    print(max_compartment)
    
    compartment_tips = compartment_dict[max_compartment]

    # Convert to python list.
    tips_list = [[t.x, t.y, t.z] for t in compartment_tips]

    # Find the convex hull of the terminal nodes contained in the compartment of interest.
    hull = ConvexHull(tips_list)

    print("The volume of the convex hull encapsulating all tip points is {} um^3".format(
        round(hull.volume, 2)))

    print("The area of the hull is {} um^2".format(round(hull.area, 2)))
    print()

    verts = []
    for v in hull.vertices:
        verts.append(compartment_tips[v])
    verts_java = ij.py.to_java(verts)
    
    verts = []
    for s in hull.simplices:
        verts.append([compartment_tips[s[0]], compartment_tips[s[1]], compartment_tips[s[2]]])
    verts_java = ij.py.to_java(verts)
    
    return verts_java

In [6]:
def run():
    
    # Fetch swc from MouseLight database by ID.
    loader = MouseLightLoader('AA1044')
    if not loader.isDatabaseAvailable():
        print("Could not connect to ML database", "Error")
        return
    if not loader.idExists():
        print("Somewhow the specified id was not found", "Error")
        return

    tree_axon = loader.getTree('axon', None)
    verts_axon = convex_hull(tree_axon)

    # Visualize the result using SNT Viewer3D.
    viewer = Viewer()
    viewer.add(tree_axon)
    axon_hull = viewer.annotateClosedSurface(verts_axon, "Axon Hull")
    axon_hull.setColor("red")
    viewer.show()

In [7]:
run()

Caudoputamen
The volume of the convex hull encapsulating all tip points is 535293333.9 um^3
The area of the hull is 3674102.62 um^2

